In [1]:
import os

import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask.dataframe as dd
import dask.array as da
from dask import delayed
from tqdm import tqdm
from dask.diagnostics import ProgressBar

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

In [5]:
def convert_parquet_to_npy(file_path, save_path):
    dataframe = pd.read_parquet(file_path)
    data_array = dataframe.to_numpy()
    np.save(save_path, data_array)

def convert_folder_to_npy(folder_path, save_folder):
    os.makedirs(save_folder, exist_ok=True) 

    files_to_convert = [file for file in os.listdir(folder_path) if file.endswith('.parquet')]
    
    for file in tqdm(files_to_convert, desc=f"Converting files in {folder_path} to .npy", unit="file"):
        file_path = os.path.join(folder_path, file)
        save_path = os.path.join(save_folder, f"{os.path.splitext(file)[0]}.npy")
        convert_parquet_to_npy(file_path, save_path)

def pad_or_truncate(data_point, target_shape):
    current_shape = data_point.shape
    if current_shape == target_shape:
        return data_point
    elif current_shape[0] < target_shape[0]:
        # Pad the data point along the first axis
        padding = [(0, target_shape[0] - current_shape[0])] + [(0, 0)] * (len(current_shape) - 1)
        return np.pad(data_point, padding, mode='constant', constant_values=0)
    else:
        # Truncate the data point along the first axis
        return data_point[:target_shape[0], ...]

def chunked_concatenate(file_paths, chunk_size, target_shape):
    chunks = []
    for chunk_start in range(0, len(file_paths), chunk_size):
        chunk_end = min(chunk_start + chunk_size, len(file_paths))
        chunk_data = [np.load(file) for file in file_paths[chunk_start:chunk_end]]
        chunks.append(chunk_data)
    return chunks

def flatten_chunks(chunks, target_shape):
    flattened_data = [pad_or_truncate(data_point, target_shape) for chunk in chunks for data_point in chunk]
    return flattened_data

def load_data(path, num_files=None, chunk_size=100, target_shape = (85199 - 10000 + 1, 20)):
    npy_train_save_folder = os.path.join(path, 'npy_train')
    npy_test_save_folder = os.path.join(path, 'npy_test')

    # Read Numpy files for train data in chunks
    train_eeg_files = [os.path.join(npy_train_save_folder, file) for file in os.listdir(npy_train_save_folder)[:num_files] if file.endswith('.npy')]
    train_eeg_chunks = chunked_concatenate(train_eeg_files, chunk_size, target_shape) if train_eeg_files else None

    # Read Numpy files for test data in chunks
    test_eeg_files = [os.path.join(npy_test_save_folder, file) for file in os.listdir(npy_test_save_folder) if file.endswith('.npy')]
    test_eeg_chunks = chunked_concatenate(test_eeg_files, chunk_size, target_shape) if test_eeg_files else None

    if train_eeg_chunks is not None:
        # Interpolate NaN values for each chunk
        for i, chunk in enumerate(train_eeg_chunks):
            train_eeg_chunks[i] = [np.nan_to_num(data_point, nan=np.nanmean(data_point)) for data_point in chunk]
            print("NaN values in train_eeg chunk {}: {}".format(i, any(np.isnan(data_point).any() for data_point in train_eeg_chunks[i])))

    # Read labels
    train_labels = pd.read_csv('train.csv', nrows=num_files)
    test_labels = pd.read_csv('test.csv')

    # Flatten the chunks to get a list of data points
    flattened_train_eeg = flatten_chunks(train_eeg_chunks, target_shape) if train_eeg_chunks else None
    flattened_test_eeg = flatten_chunks(test_eeg_chunks, target_shape) if test_eeg_chunks else None

    return flattened_train_eeg, flattened_test_eeg, train_labels, test_labels


In [7]:
# run this once
convert_folder_to_npy('./data/data/train_eegs', 'data/npy_train')
convert_folder_to_npy('./data/data/test_eegs', 'data/npy_test')

Converting files in ./data/data/train_eegs to .npy: 100%|██████████| 24/24 [00:02<00:00,  8.97file/s]
Converting files in ./data/data/test_eegs to .npy: 100%|██████████| 1/1 [00:00<00:00, 100.33file/s]


In [8]:
flattened_train_eeg, flattened_test_eeg, train_labels, test_labels = load_data('data', num_files=100, chunk_size=50)

NaN values in train_eeg chunk 0: False


In [21]:
labels=pd.read_csv('C:/Users/Dell/Desktop/EEG-Code/mini_data/mini_data/labels.csv')

In [22]:
labels

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0,0,0,3,0,0
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0,0,0,3,0,0
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0,0,0,3,0,0
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0,0,0,3,0,0


In [31]:
labels['expert_consensus'].unique()

array(['Seizure', 'GPD', 'LRDA', 'Other', 'GRDA', 'LPD'], dtype=object)

*using mini_data*

In [35]:
import numpy as np
import pandas as pd
import os

# Load labels from CSV
labels_path = './mini_data/mini_data/labels.csv'
labels_df = pd.read_csv(labels_path)

# 'eeg_id' is the column containing file names
eeg_ids = labels_df['eeg_id'].tolist()

# Load EEG data from .npy files
eeg_data = []
eeg_folder_path = './mini_data/mini_data/train_eeg'  

for eeg_id in eeg_ids:
    file_path = os.path.join(eeg_folder_path, f'{eeg_id}.npy')
    if os.path.exists(file_path):
        data = np.load(file_path)
        eeg_data.append(data)

eeg_data = np.array(eeg_data)

